Creacion de una Base de Datos con SQLite

Vamos a crear una base de datos para guardar nuestro dataset y hacerlo consumible para Power BI. Usaremos SQLite como base de datos, ya que es fácil de usar y no requiere configuración adicional.

In [7]:
import pandas as pd
import sqlite3

def cargar_datos():
    df_velocidad = pd.read_excel('../6._DataSets_Limpio/df_internet_limpio.xlsx', sheet_name='Velocidad')
    df_acceso_tecnologia = pd.read_excel('../6._DataSets_Limpio/df_internet_limpio.xlsx', sheet_name='Acceso_Tecnologia')
    df_penetracion_poblacion = pd.read_excel('../6._DataSets_Limpio/df_internet_limpio.xlsx', sheet_name='Penetracion_Poblacion')
    df_penetracion_hogares = pd.read_excel('../6._DataSets_Limpio/df_internet_limpio.xlsx', sheet_name='Penetracion_Hogares')
    df_accesos_velocidad = pd.read_excel('../6._DataSets_Limpio/df_internet_limpio.xlsx', sheet_name='Accesos_Velocidad')
    df_ingresos = pd.read_excel('../6._DataSets_Limpio/df_internet_limpio.xlsx', sheet_name='Ingresos')
    
    # Función para limpiar y convertir fechas
    def limpiar_fecha(x):
        if pd.isna(x):
            return pd.NaT
        try:
            return pd.to_datetime(x, errors='coerce')
        except ValueError:
            return pd.NaT
    
    # Función para limpiar y convertir trimestres
    def limpiar_trimestre(x):
        if pd.isna(x):
            return 0
        try:
            return int(x)
        except ValueError:
            return 0
    
    # Aplicar las funciones de limpieza a cada DataFrame
    for df in [df_velocidad, df_acceso_tecnologia, df_penetracion_poblacion, df_penetracion_hogares, df_accesos_velocidad, df_ingresos]:
        df['Año'] = df['Año'].apply(limpiar_fecha)
        df['Trimestre'] = df['Trimestre'].apply(limpiar_trimestre)
    
    return df_velocidad, df_acceso_tecnologia, df_penetracion_poblacion, df_penetracion_hogares, df_accesos_velocidad, df_ingresos

def crear_base_datos_sqlite():
    conn = sqlite3.connect('internet_argentina.db')
    cursor = conn.cursor()
    
    # Crear tablas
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS velocidad
        (id INTEGER PRIMARY KEY AUTOINCREMENT,
         anio INT,
         trimestre INT,
         provincia TEXT,
         velocidad_media REAL)
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS acceso_tecnologia
        (id INTEGER PRIMARY KEY AUTOINCREMENT,
         anio INT,
         trimestre INT,
         provincia TEXT,
         adsl REAL,
         cablemodem REAL,
         fibra_optica REAL,
         total_tecnologia REAL)
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS penetracion_poblacion
        (id INTEGER PRIMARY KEY AUTOINCREMENT,
         anio INT,
         trimestre INT,
         provincia TEXT,
         penetracion REAL)
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS penetracion_hogares
        (id INTEGER PRIMARY KEY AUTOINCREMENT,
         anio INT,
         trimestre INT,
         provincia TEXT,
         penetracion REAL)
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS accesos_velocidad
        (id INTEGER PRIMARY KEY AUTOINCREMENT,
         anio INT,
         trimestre INT,
         provincia TEXT,
         velocidad REAL)
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ingresos
        (id INTEGER PRIMARY KEY AUTOINCREMENT,
         anio INT,
         trimestre INT,
         provincia TEXT,
         ingresos REAL)
    ''')
    
    conn.commit()
    return conn

def guardar_datos_en_sqlite(conn, df_velocidad, df_acceso_tecnologia, df_penetracion_poblacion, df_penetracion_hogares, df_accesos_velocidad, df_ingresos):
    # Guardar datos de velocidad
    df_velocidad.to_sql('velocidad', conn, if_exists='replace', index=False)
    
    # Guardar datos de acceso tecnología
    df_acceso_tecnologia.to_sql('acceso_tecnologia', conn, if_exists='replace', index=False)
    
    # Guardar datos de penetración población
    df_penetracion_poblacion.to_sql('penetracion_poblacion', conn, if_exists='replace', index=False)
    
    # Guardar datos de penetración hogares
    df_penetracion_hogares.to_sql('penetracion_hogares', conn, if_exists='replace', index=False)
    
    # Guardar datos de accesos velocidad
    df_accesos_velocidad.to_sql('accesos_velocidad', conn, if_exists='replace', index=False)
    
    # Guardar datos de ingresos
    df_ingresos.to_sql('ingresos', conn, if_exists='replace', index=False)
    
    conn.commit()

def visualizar_base_datos():
    # Conectar a la base de datos
    conn = sqlite3.connect('internet_argentina.db')
    
    # Crear un cursor
    cursor = conn.cursor()
    
    # Obtener los nombres de las tablas
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablas = cursor.fetchall()
    
    # Cerrar la conexión
    conn.close()

def main():
    # Cargar datos
    df_velocidad, df_acceso_tecnologia, df_penetracion_poblacion, df_penetracion_hogares, df_accesos_velocidad, df_ingresos = cargar_datos()
    
    # Crear base de datos SQLite
    conn = crear_base_datos_sqlite()
    
    # Guardar datos en la base de datos
    guardar_datos_en_sqlite(conn, df_velocidad, df_acceso_tecnologia, df_penetracion_poblacion, df_penetracion_hogares, df_accesos_velocidad, df_ingresos)
    
    # Cerrar la conexión
    conn.close()

    print("Datos guardados exitosamente en la base de datos SQLite 'internet_argentina.db'")

    # Visualizar la base de datos
    visualizar_base_datos()

if __name__ == "__main__":
    main()

Datos guardados exitosamente en la base de datos SQLite 'internet_argentina.db'
